In [150]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pickle
from sklearn.pipeline import make_pipeline
from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score,mean_squared_error
from sklearn.manifold import TSNE

import os,cv2,math,glob,random
from math import cos, sin


In [151]:
# connecting to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [152]:
# Reading the whole data
df_raw = pd.read_csv('/content/drive/MyDrive/AFLW2000/whole_data_normalized.csv')
df_raw

,0,1,2,3,4,5,6,7,8,9,...,929,930,931,932,933,934,935,936,937,938
0,-0.057177,0.324858,-0.002146,0.080703,-0.048256,0.151521,-0.097847,-0.165961,0.001505,0.004881,...,-0.341429,0.015144,-0.329294,0.348905,-0.399302,0.383050,-0.432656,0.027256,-0.247779,-0.010263
1,0.194735,0.272097,0.028134,0.067446,0.171773,0.098387,0.022502,-0.180925,0.003351,0.003668,...,-0.442275,0.258387,-0.410401,0.631726,-0.537775,0.672187,-0.582089,-0.446169,0.684181,-0.270866
2,0.044035,0.397133,0.007734,0.091990,0.030889,0.205728,-0.056760,-0.147080,0.006657,0.006530,...,-0.270760,0.236964,-0.266185,0.801520,-0.273566,0.862289,-0.337867,0.093373,0.062462,-0.125822
3,0.062973,0.282248,0.014461,0.076476,0.044149,0.130072,-0.052850,-0.167922,0.001940,0.006611,...,-0.394488,0.112805,-0.374413,0.478794,-0.515546,0.505729,-0.550703,-0.241359,0.205732,-0.150232
4,-0.066651,0.292848,-0.005003,0.076158,-0.070066,0.131578,-0.096194,-0.163922,0.002696,0.007060,...,-0.361601,-0.042270,-0.345408,0.217688,-0.440829,0.241148,-0.458950,-0.167856,-0.361028,0.058591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,0.001562,0.240536,0.004970,0.068933,0.003152,0.113671,-0.060087,-0.168740,0.002330,0.001066,...,-0.375928,0.100641,-0.357247,0.474052,-0.472996,0.504516,-0.500678,-0.199835,-0.007556,-0.035916
1849,-0.216270,0.131494,-0.045367,0.042140,-0.109332,0.030021,0.068064,-0.179301,0.004354,0.005374,...,-0.270592,0.281402,-0.257429,0.572243,-0.122748,0.611529,-0.123718,-0.304740,0.074078,0.735418
1850,-0.081104,0.295605,-0.013374,0.079067,-0.056509,0.139676,-0.056976,-0.171710,0.002207,0.007331,...,-0.320076,0.097087,-0.310236,0.456003,-0.311169,0.491758,-0.338711,-0.329202,-0.237305,0.184649
1851,0.020203,0.377361,0.007438,0.087908,0.017490,0.183275,-0.042354,-0.163552,0.008220,0.006576,...,-0.303996,0.183359,-0.298560,0.654073,-0.327672,0.695340,-0.364091,-0.019941,0.076776,0.011841


In [153]:
# Sparating features from labels
features = df_raw.iloc[:,:936]
features

,0,1,2,3,4,5,6,7,8,9,...,926,927,928,929,930,931,932,933,934,935
0,-0.057177,0.324858,-0.002146,0.080703,-0.048256,0.151521,-0.097847,-0.165961,0.001505,0.004881,...,0.049427,-0.358898,0.022385,-0.341429,0.015144,-0.329294,0.348905,-0.399302,0.383050,-0.432656
1,0.194735,0.272097,0.028134,0.067446,0.171773,0.098387,0.022502,-0.180925,0.003351,0.003668,...,0.333681,-0.477440,0.295615,-0.442275,0.258387,-0.410401,0.631726,-0.537775,0.672187,-0.582089
2,0.044035,0.397133,0.007734,0.091990,0.030889,0.205728,-0.056760,-0.147080,0.006657,0.006530,...,0.332858,-0.284174,0.274349,-0.270760,0.236964,-0.266185,0.801520,-0.273566,0.862289,-0.337867
3,0.062973,0.282248,0.014461,0.076476,0.044149,0.130072,-0.052850,-0.167922,0.001940,0.006611,...,0.171898,-0.420038,0.135732,-0.394488,0.112805,-0.374413,0.478794,-0.515546,0.505729,-0.550703
4,-0.066651,0.292848,-0.005003,0.076158,-0.070066,0.131578,-0.096194,-0.163922,0.002696,0.007060,...,-0.027838,-0.382886,-0.045075,-0.361601,-0.042270,-0.345408,0.217688,-0.440829,0.241148,-0.458950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,0.001562,0.240536,0.004970,0.068933,0.003152,0.113671,-0.060087,-0.168740,0.002330,0.001066,...,0.155809,-0.398523,0.120006,-0.375928,0.100641,-0.357247,0.474052,-0.472996,0.504516,-0.500678
1849,-0.216270,0.131494,-0.045367,0.042140,-0.109332,0.030021,0.068064,-0.179301,0.004354,0.005374,...,0.337438,-0.272494,0.301758,-0.270592,0.281402,-0.257429,0.572243,-0.122748,0.611529,-0.123718
1850,-0.081104,0.295605,-0.013374,0.079067,-0.056509,0.139676,-0.056976,-0.171710,0.002207,0.007331,...,0.147547,-0.330161,0.111843,-0.320076,0.097087,-0.310236,0.456003,-0.311169,0.491758,-0.338711
1851,0.020203,0.377361,0.007438,0.087908,0.017490,0.183275,-0.042354,-0.163552,0.008220,0.006576,...,0.264125,-0.314371,0.213279,-0.303996,0.183359,-0.298560,0.654073,-0.327672,0.695340,-0.364091


In [167]:
# Extracting
labels = df_raw.iloc[:,936:]
pitch_raw = labels.iloc[:,0]
yaw_raw = labels.iloc[:,1]
roll_raw = labels.iloc[:,2]


In [155]:
labels.describe()

,936,937,938
count,1853.000000,1853.000000,1853.000000
mean,-0.095366,0.013611,-0.033167
std,0.549292,0.580266,0.649282
min,-12.457202,-6.130066,-16.969362
25%,-0.246453,-0.256714,-0.160274
50%,-0.123371,0.019403,-0.017463
75%,0.028133,0.305791,0.126460
max,15.598820,3.277500,2.166107


In [156]:
# Spliting data 
x_train, x_test, y_train_pitch, y_test_pitch = train_test_split(features, pitch_raw, test_size=0.2,shuffle=True ,random_state=42)
x_train, x_test, y_train_yaw, y_test_yaw = train_test_split(features, yaw_raw, test_size=0.2, shuffle=True, random_state=42)
x_train, x_test, y_train_roll, y_test_roll = train_test_split(features, roll_raw, test_size=0.2,shuffle=True, random_state=42)


In [178]:
pipeline_yaw = make_pipeline(StandardScaler(),PCA(n_components=0.99),())
pipeline_pitch = make_pipeline(StandardScaler(),PCA(n_components=0.99),RandomForestRegressor())
pipeline_roll = make_pipeline(StandardScaler(),PCA(n_components=0.99),RandomForestRegressor())

In [179]:

model_pitch = pipeline_pitch.fit(x_train,y_train_pitch)
model_yaw = pipeline_yaw.fit(x_train,y_train_yaw)
model_roll = pipeline_roll.fit(x_train,y_train_roll)


In [182]:
print(f'Pitch SCORES are:')
print(f'Mean Square Error is {mean_squared_error(y_train_pitch,model_pitch.predict(x_train))}')
print(f'R2 Score is: {r2_score(y_test_pitch,model_pitch.predict(x_test))}')
print(f'R2 Score "Train" is: {r2_score(y_train_pitch,model_pitch.predict(x_train))}')

print('=========================================')

print(f'Yaw SCORES are:')
print(f'Mean Square Error is {mean_squared_error(y_train_yaw,model_yaw.predict(x_train))}')
print(f'R2 Score is: {r2_score(y_test_yaw,model_yaw.predict(x_test))}')
print(f'R2 Score "Train" is: {r2_score(y_train_yaw,model_yaw.predict(x_train))}')

print('===========================================')

print(f'Roll SCORES are:')
print(f'Mean Square Error is {mean_squared_error(y_train_roll,model_roll.predict(x_train))}')
print(f'R2 Score is: {r2_score(y_test_roll,model_roll.predict(x_test))}')
print(f'R2 Score "Train" is: {r2_score(y_train_roll,model_roll.predict(x_train))}')


Pitch SCORES are:
Mean Square Error is 0.08417088140433042
R2 Score is: 0.1640824236842473
R2 Score "Train" is: 0.7581359438413511
Yaw SCORES are:
Mean Square Error is 0.008796401430464493
R2 Score is: 0.8287376093448644
R2 Score "Train" is: 0.9743531273195322
Roll SCORES are:
Mean Square Error is 0.05011614744917167
R2 Score is: -0.5066934640554621
R2 Score "Train" is: 0.8973362064352032


In [183]:
with open('/content/drive/MyDrive/AFLW2000/model_pitch.pkl', 'wb') as f:
    pickle.dump(model_pitch, f)

with open('/content/drive/MyDrive/AFLW2000/model_yaw.pkl', 'wb') as f:
    pickle.dump(model_yaw, f)

with open('/content/drive/MyDrive/AFLW2000/model_roll.pkl', 'wb') as f:
    pickle.dump(model_roll, f)

